In [1]:
from filterpy.kalman import UnscentedKalmanFilter as UKF
from filterpy.kalman import MerweScaledSigmaPoints

import numpy as np
import sympy as sp
import math
import matplotlib.pyplot as plt
%matplotlib qt
DEG_TO_RAD = math.pi/180
RAD_TO_DEG = 180/math.pi
gravitation = 9.81
l = 0.5

In [2]:
def polar_to_kartesian(r, theta, phi):
    x = r * np.sin(theta) * np.cos(phi)
    y = r * np.sin(theta) * np.sin(phi)
    z = -r * np.cos(theta)
    return [x,y,z]

X ist der Zustand $ \vec x = \begin{bmatrix} \theta \\ \phi \\ \theta' \\ \phi' \end{bmatrix} $ wobei gilt $ q = \begin{bmatrix} \theta \\ \phi \end{bmatrix} $


Die Funktion **transfer_function** berechnet $\dot{ \vec x}$

In [3]:
def state_first_deriv(x):
    dq = x[2:]
    dq2 = second_deriv_theta_phi(x)
    
    return np.concatenate([dq, dq2])

In [12]:
def second_deriv_theta_phi(x):
    theta, phi, d_theta, d_phi = x
    c, s, t = np.cos(theta), np.sin(theta), np.tan(theta)

    d2_theta = (d_phi**2 * c - gravitation / l) * s
    d2_phi = -2 * d_theta * d_phi / t
    if (t == 0):
        print ('t is 0')

    return np.array([d2_theta, d2_phi])
    

In [13]:
def f(x, dt):
    x_new = x + state_first_deriv(x) * dt
    return x_new
    

In [7]:
# Nice divisors are divisors (2.5,4.2), (4, 6), (5, 2) and (1.1, inf)

x = [[np.pi / 1.1, 0, 0, 0]]
#x = [[ 45* DEG_TO_RAD, 0, 0 * DEG_TO_RAD, 0]]
positions = []
Ts = [0]
dt = 1/100000
N = 2000000
for i in range(N):
    positions.append(polar_to_kartesian(l, x[i][0],x[i][1]))
    x.append(f(x[i],dt))
    Ts.append(i*dt)

## Plot 

In [9]:
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d.axes3d as p3
import matplotlib.animation as animation

In [10]:
positions = np.array(positions)


Plot Unanimated

In [11]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.set_xlim3d(-1, 1)
ax.set_ylim3d(-1, 1)
ax.set_zlim3d(-1, 1)
ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_zlabel("Z")

string = plt.quiver(0,0,0,0,0,0)

# plotting the overall curve
#ax.scatter(positions[:,0], positions[:,1], positions[:,2], c='b', marker='.', linewidth=0.00001)

def func(num, positions, pendulum): 
    x = positions[num, 0]
    y = positions[num, 1]
    z = positions[num, 2]
    
    pendulum.set_data(x,y)
    pendulum.set_3d_properties(z)
    
    
    u = positions[num, 0]
    v = positions[num, 1]
    w = positions[num, 2]
    
    global string
    string.remove()
    string = plt.quiver(0,0,0, u, v, w,color='g')
    
pendulum, = plt.plot([positions[0,0]], [positions[0,1]], [positions[0,2]], c='r', marker='o')

ani = animation.FuncAnimation(fig, func, fargs=(positions, pendulum), interval=1/1000, blit=False)

ax.plot(positions[:,0], positions[:,1], positions[:,2], linewidth=0.6)